In [ ]:
from ctraptools.kymos.detect import Detector
from lumicks import pylake

import ctraptools.kymos.stepfinding as ksf
import ctraptools.kymos.io as kio
import os

In [ ]:
# Parameters
path = ""
channel = 0

In [ ]:
# Loading kymograph as image
if os.path.splitext(path)[1] == ".tif":
    image = kio.read_image(path,channel,x_range=None)
elif os.path.splitext(path)[1] == ".h5" or os.path.splitext(path)[1] == ".hdf5":
    file = pylake.File(path)
    kymo = next(iter(file.kymos.values()))
    image = kymo.get_image()[:,:,channel]

rootpath = os.path.dirname(path)+'/'
rootname = os.path.basename(os.path.splitext(path)[0])

In [ ]:
# (Optional) Cropping the image in the x-axis
# image = image[:,12:400]

In [ ]:
# Initialising Detector
half_t_w = 0
detector = Detector(half_t_w=half_t_w,peak_det_thresh = 0.3, n_max=10, c_lb=2, c_ub=7, c_def=3, max_dist=15,min_track_length = 10, track_heritage_weight=100, )

In [ ]:
# (Optional) Testing fit parameters
fr = 10
peaks = detector.fit_peaks(image,fr)
kio.plot_gauss_for_frame(peaks,fr,image,half_t_w=half_t_w)

In [ ]:
# Detect peaks and track
tracks = detector.detect(image)

In [ ]:
# Measuring intensity
for track in tracks.values():
    track.measure_intensity(image)

# Step fitting (to be added later)
# ksf.fit_steps(tracks,image)

# Saving files
kio.save_overlay(tracks,image,rootpath+rootname+'_C'+str(channel)+"_overlay")
kio.write_peak_traces(tracks,rootpath+rootname+'_C'+str(channel)+"_traces")
kio.save_plots(tracks,rootpath+rootname+'_C'+str(channel)+"_traces")
kio.write_change_points(tracks,rootpath+rootname+'_C'+str(channel)+'_steps')